In [22]:
from __future__ import division
import six
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
import re
import numpy as np
import csv
import pandas as pd
from pandas import read_csv
import sys

In [23]:
GOOGLE_APPLICATION_CREDENTIALS = "google_credentials.json"

In [24]:
def clean_tweet(tweet): 
    ''' 
    Utility function to clean tweet text by removing links, special characters 
    using simple regex statements. 
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

In [27]:
def get_tweets(twitter_handle):
    tweet_data = read_csv('..//Tweets//' + twitter_handle + '_tweets.csv', encoding = 'utf-8')
    tweet_data = tweet_data.dropna()
    return tweet_data

In [32]:
tweets = get_tweets('ewarren')

In [37]:
def word_in_text(word, text):
    word = word.lower()
    text = text.lower()
    match = re.search(word, text)
    if match:
        return True
    return False

def word_basket_in_text(word_list, text):
    status = True
    for word in word_list:
        if type(word)==list:
            list_status = False
            for list_word in word:
                list_status = list_status or word_in_text(list_word, text)
            status = status and list_status
        else: status = status and word_in_text(word, text)
    return status

In [38]:
Healthcare = [u'care','affordable care act', 'medicare', 'obamacare',  'benefits', 'opioid', 'abortion', 'medicaid', 'single payer', 'medication', 'prescription', 'drugs', 'hospital', 'health', 'care']
Terrorism = [u'al', u'terror', u'sanctions', u'drone', u'oil', u'gulf', u'intelligence', u'ISIS', u'military', u'nuclear']
Economy = [u'china', u'tariff', u'mexico', u'fed', u'sec', u'interest', u'rate', u'immigration', u'debt', u'unemployment', u'growth', u'inflation', u'trade war', u'dollar', u'gdp', u'g20', u'recession', u'stock', u'market']
lgbtq = [u'gay', u'lesbian', u'pride', u'community', u'lgbt', u'same-sex', u'marriage', u'orientation', u'parade', u'stonewall', u'riot', u'civil', u'homosexual', u'queer', u'conversion']
medicare = ['MedicareForAll', ['Medicare']] 


In [39]:
healthcare_warren = []
for tweet in tweets['text']:
    if word_basket_in_text(medicare, tweet):
        healthcare_warren.append(tweet)

In [40]:
healthcare_warren = list(set(healthcare_warren))

In [42]:
def implicit():
    from google.cloud import storage

    # If you don't specify credentials when constructing the client, the
    # client library will look for credentials in the environment.
    storage_client = storage.Client()

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)

In [43]:
client = language.LanguageServiceClient.from_service_account_json(GOOGLE_APPLICATION_CREDENTIALS)

text = healthcare_warren[4]
                                                                
if isinstance(text, six.binary_type):
    text = text.decode('utf-8')

document = types.Document(
    content=text.encode('utf-8'),
    type=enums.Document.Type.PLAIN_TEXT)

# Detect and send native Python encoding to receive correct word offsets.
encoding = enums.EncodingType.UTF32
if sys.maxunicode == 65535:
    encoding = enums.EncodingType.UTF16

result = client.analyze_entity_sentiment(document, encoding)

for entity in result.entities:
    print('Mentions: ')
    print(u'Name: "{}"'.format(entity.name))
    for mention in entity.mentions:
        print(u'  Begin Offset : {}'.format(mention.text.begin_offset))
        print(u'  Content : {}'.format(mention.text.content))
        print(u'  Magnitude : {}'.format(mention.sentiment.magnitude))
        print(u'  Sentiment : {}'.format(mention.sentiment.score))
        print(u'  Type : {}'.format(mention.type))
    print(u'Salience: {}'.format(entity.salience))
    print(u'Sentiment: {}\n'.format(entity.sentiment))

Mentions: 
Name: "emergency"
  Begin Offset : 67
  Content : one
  Magnitude : 0.10000000149011612
  Sentiment : 0.10000000149011612
  Type : 2
  Begin Offset : 103
  Content : emergency
  Magnitude : 0.0
  Sentiment : 0.0
  Type : 2
Salience: 0.34767287969589233
Sentiment: magnitude: 0.20000000298023224
score: 0.10000000149011612


Mentions: 
Name: "chunk"
  Begin Offset : 14
  Content : chunk
  Magnitude : 0.20000000298023224
  Sentiment : 0.20000000298023224
  Type : 2
Salience: 0.20815634727478027
Sentiment: magnitude: 0.20000000298023224
score: 0.20000000298023224


Mentions: 
Name: "life"
  Begin Offset : 26
  Content : life
  Magnitude : 0.5
  Sentiment : 0.5
  Type : 2
Salience: 0.20815634727478027
Sentiment: magnitude: 0.5
score: 0.5


Mentions: 
Name: "families"
  Begin Offset : 44
  Content : families
  Magnitude : 0.30000001192092896
  Sentiment : -0.30000001192092896
  Type : 2
Salience: 0.09234125167131424
Sentiment: magnitude: 0.30000001192092896
score: -0.30000001192092

In [46]:
result

entities {
  name: "emergency"
  type: OTHER
  salience: 0.34767287969589233
  mentions {
    text {
      content: "one"
      begin_offset: 67
    }
    type: COMMON
    sentiment {
      magnitude: 0.10000000149011612
      score: 0.10000000149011612
    }
  }
  mentions {
    text {
      content: "emergency"
      begin_offset: 103
    }
    type: COMMON
    sentiment {
    }
  }
  sentiment {
    magnitude: 0.20000000298023224
    score: 0.10000000149011612
  }
}
entities {
  name: "chunk"
  type: OTHER
  salience: 0.20815634727478027
  mentions {
    text {
      content: "chunk"
      begin_offset: 14
    }
    type: COMMON
    sentiment {
      magnitude: 0.20000000298023224
      score: 0.20000000298023224
    }
  }
  sentiment {
    magnitude: 0.20000000298023224
    score: 0.20000000298023224
  }
}
entities {
  name: "life"
  type: OTHER
  salience: 0.20815634727478027
  mentions {
    text {
      content: "life"
      begin_offset: 26
    }
    type: COMMON
    sentiment 